###  IBM Cloud Utility notebook: browse service instances

Use this notebook to display the list of IBM Cloud service instances you have access to.
To run the notebook: <br>
 * Add your IBM Cloud API key or your IBM Cloud user ID and password to the next cell.
 * Run the entire notebook (**Cell** > **Run all**) without making any other changes.
 * Scroll to the bottom of the notebook.

In [ ]:
#@ hidden_cell
# Specify IBM Cloud API key (recommended)
# api_key = None

# ... or a user id/password
id = None
pw = None

In [ ]:
import json
import pandas as pd
import pixiedust
import requests
import urllib

In [ ]:
API_base_URL = 'https://api.ng.bluemix.net{}'
debug = False

#### Obtain access token

In [ ]:
if api_key is not None:
    id = 'apikey'
    pw = api_key

response = requests.get(API_base_URL.format('/info'))
results = response.json()
auth_endpoint = results['authorization_endpoint'] + '/oauth/token'

data = 'grant_type=password&username={0}&password={1}'.format(id, pw)
auth = ('cf', '')
headers = {
    'accept': 'application/json',
    'content-type': 'application/x-www-form-urlencoded;charset=utf-8'
    }
response = requests.post(auth_endpoint, data=data, headers=headers, auth=auth)
if response.status_code == 200:
    results = response.json()
    authorization = results['token_type'] + ' ' + results['access_token']
    print('Got OAuth token')
else:
    print('Fatal error obtaining authentication token: {}. Did you specify a valid IBM Cloud API key or IBM Cloud credentials?'.format(response))


In [ ]:
url = API_base_URL.format('/v2/organizations')

http_headers = {
    'accept': 'application/json',
    'content-type': 'application/json',
    'authorization': authorization
    }
response = requests.get(url, headers=http_headers)
if response.status_code == 200:
    orgs = []
    for org in json.loads(response.text)['resources']:
        orgs.append(org['entity']['name'])   
    orgs = sorted(orgs, key=lambda s: s.lower())
    print('You have access to the following organizations: {}'.format(orgs))
    service_plans = None
else:
    print('Fatal error retrieving organizations: {}'.format(response))

In [ ]:
org_lookup = {}

def fetch_organization_list(url):
    http_headers = {
        'accept': 'application/json',
        'content-type': 'application/json',
        'authorization': authorization
        }
    if debug:
        print('GET {}'.format(url))
    response = requests.get(API_base_URL.format(url), headers=http_headers)
    if response.status_code == 200:
        for resource in response.json().get('resources', []):
            org_lookup[resource['metadata']['guid']] = resource['entity']['name']          
        return response.json()['next_url']
    else:
        print('Fatal error retrieving organization list: {}'.format(response))
        return None

# https://apidocs.cloudfoundry.org/280/services/list_all_services.html
url = '/v2/organizations?results-per-page=100'

while url is not None:
    print('Populating organization lookup...')
    url = fetch_organization_list(url)
    
print('Found {} organizations.'.format(len(org_lookup)))   

In [ ]:
space_lookup = {}

def fetch_space_list(url):
    http_headers = {
        'accept': 'application/json',
        'content-type': 'application/json',
        'authorization': authorization
        }
    if debug:
        print('GET {}'.format(url))
    response = requests.get(API_base_URL.format(url), headers=http_headers)
    if response.status_code == 200:
        for resource in response.json().get('resources', []):
                space_lookup[resource['metadata']['guid']] = {
                    'space_name': resource['entity']['name'],
                    'org_guid': resource['entity']['organization_guid'],
                    'org_name': org_lookup[resource['entity']['organization_guid']]
                }
        return response.json()['next_url']
    else:
        print('Fatal error retrieving organization list: {}'.format(response))
        return None

# https://apidocs.cloudfoundry.org/280/organizations/list_all_spaces_for_the_organization.html


for org_guid in org_lookup.keys(): 
    url = '/v2/organizations/{}/spaces?results-per-page=100'.format(org_guid)
    while url is not None:
        print('Searching for spaces in org {}...'.format(org_lookup[org_guid]))
        url = fetch_space_list(url)
        
del org_guid        
    
print('Found {} spaces.'.format(len(space_lookup)))   

In [ ]:
service_lookup = {}

def fetch_complete_service_list(url):
    http_headers = {
        'accept': 'application/json',
        'content-type': 'application/json',
        'authorization': authorization
        }
    if debug:
        print('GET {}'.format(url))
    response = requests.get(API_base_URL.format(url), headers=http_headers)
    if response.status_code == 200:
        for resource in response.json().get('resources', []):
            service_lookup[resource['metadata']['guid']] = resource['entity']['label']          
        return response.json()['next_url']
    else:
        print('Fatal error retrieving service list: {}'.format(response))
        return None

# https://apidocs.cloudfoundry.org/280/services/list_all_services.html
url = '/v2/services?results-per-page=100'

while url is not None:
    print('Populating service lookup...')
    url = fetch_complete_service_list(url)
    
print('Found {} services.'.format(len(service_lookup)))    

In [ ]:
service_plan_lookup = {}

def fetch_service_plan_list(url):
    http_headers = {
        'accept': 'application/json',
        'content-type': 'application/json',
        'authorization': authorization
        }
    if debug:
        print('GET {}'.format(url))
    response = requests.get(API_base_URL.format(url), headers=http_headers)
    if response.status_code == 200:
        for resource in response.json().get('resources', []):
            service_plan_lookup[resource['metadata']['guid']] = resource['entity']['name']          
        return response.json()['next_url']                
    else:
        print('Fatal error retrieving service plan information: {}'.format(response))
        return None

# https://apidocs.cloudfoundry.org/280/service_plans/list_all_service_plans.html
url = '/v2/service_plans?results-per-page=100'

while url is not None:
    print('Loading service plan information...')
    url = fetch_service_plan_list(url)
    
print('Found {} service plans.'.format(len(service_plan_lookup)))    


In [ ]:
service_instances = []

def fetch_service_instance_list(url):
    http_headers = {
        'accept': 'application/json',
        'content-type': 'application/json',
        'authorization': authorization
        }
    
    if debug:
        print('GET {}'.format(url))
    response = requests.get(API_base_URL.format(url), headers=http_headers)
    if response.status_code == 200:
        for resource in response.json().get('resources', []):
                service = {'service_instance_name':resource['entity']['name'],
                           'service_guid':resource['entity']['service_guid'],
                           'created_at':resource['metadata']['created_at'],
                           'plan_guid': resource['entity'].get('service_plan_guid', None),
                           'space_guid': resource['entity'].get('space_guid', None)}
                
                if space_lookup is not None and space_lookup.get(service['space_guid'], None) is not None:
                    service['space_name'] = space_lookup[service['space_guid']]['space_name']
                    service['org_name'] = space_lookup[service['space_guid']]['org_name']
                    service['org_guid'] = space_lookup[service['space_guid']]['org_guid']
                        
                if service_lookup is not None and service_lookup.get(service['service_guid'], None) is not None:
                    service['service_name'] = service_lookup[service['service_guid']]
                else:
                    service['service_name'] = None                
                if service_plan_lookup is not None and service_plan_lookup.get(service['plan_guid'], None) is not None:
                    service['service_plan_name'] = service_plan_lookup[service['plan_guid']]
                else:
                    service['service_plan_name'] = None
                service_instances.append(service)
        return response.json()['next_url']  
    else:
        print('Fatal error retrieving service instance information: {}'.format(response))
        return None                

# https://apidocs.cloudfoundry.org/280/service_instances/list_all_service_instances.html
url = '/v2/service_instances?results-per-page=100'

while url is not None:
    print('Loading service instance information...')
    url = fetch_service_instance_list(url)
    
print('Found {} service instances.'.format(len(service_instances)))   



In [ ]:
# create Pandas DataFrame
services_df = pd.DataFrame(service_instances)

### Explore IBM Cloud services

The app lists all service instances that the specified IBM Cloud API key or IBM Cloud user id + password has access to.

In [ ]:
from pixiedust.display.app import *

@PixieApp
@Logger()
class FetchServiceCredentials():
    
    @route()
    @templateArgs 
    def main_screen(self):
        
        self.debug("Entering method main screen")
        
        self.services_df = services_df
        self.services_df['_is_hidden'] = False
        self.service_types_df = self.services_df[['service_guid', 'service_name']].drop_duplicates().sort_values(by=['service_name'])
        return """
            <!-- custom styling -->
            <style>
                div.outer-wrapper {
                    display: table;width:100%;height:100px;
                }
                div.inner-wrapper {
                    display: table-cell;vertical-align: middle;height: 100%;width: 100%;
                }
                th { text-align:center; }
                td { text-align: left; }
            </style>

        <div class="outer-wrapper">

            <select id="service_guid_filter{{prefix}}" 
                    pd_target="matching_service_list{{prefix}}"
                    pd_options="service_guid=$val(service_guid_filter{{prefix}});service_plan_guid=$val(service_plan_guid_filter{{prefix}});org_guid=$val(org_guid_filter{{prefix}});space_guid=$val(space_guid_filter{{prefix}})">

               <option value="---ALL--">All service types</option>
             {%for service_type in this.service_types_df.itertuples()%}
               <option value="{{service_type['service_guid']}}">{{service_type['service_name']}}</option>
             {%endfor%}
            </select>
            
            <select id="service_plan_guid_filter{{prefix}}" 
                    pd_target="matching_service_list{{prefix}}"
                    pd_options="service_guid=$val(service_guid_filter{{prefix}});service_plan_guid=$val(service_plan_guid_filter{{prefix}});org_guid=$val(org_guid_filter{{prefix}});space_guid=$val(space_guid_filter{{prefix}})">

               <option value="---ALL--">All service plans</option>
            </select>
            
            <select id="org_guid_filter{{prefix}}" 
                    pd_target="matching_service_list{{prefix}}"
                    pd_options="service_guid=$val(service_guid_filter{{prefix}});service_plan_guid=$val(service_plan_guid_filter{{prefix}});org_guid=$val(org_guid_filter{{prefix}});space_guid=$val(space_guid_filter{{prefix}})">

               <option value="---ALL--">All organizations</option>
            </select>
            
            <select id="space_guid_filter{{prefix}}" 
                    pd_target="matching_service_list{{prefix}}"
                    pd_options="service_guid=$val(service_guid_filter{{prefix}});service_plan_guid=$val(service_plan_guid_filter{{prefix}});org_guid=$val(org_guid_filter{{prefix}});space_guid=$val(space_guid_filter{{prefix}})">

               <option value="---ALL--">All spaces</option>
            </select>

        </div>   
        
        <div class="outer-wrapper" id="matching_service_list{{prefix}}">

        </div>
        
"""
    
   
    @route(service_guid="*", service_plan_guid="*", org_guid="*", space_guid="*")
    def apply_filter(self,service_guid, service_plan_guid, org_guid, space_guid):
        
        self.debug("Entering method apply_filter: {} {} {} {}".format(service_guid, service_plan_guid, org_guid, space_guid))
        
        # if no specific service_guid, service_plan_guid, org_guid or space_guid was selected, set the corresponding variable to None
        if service_guid == "---ALL--":
            service_guid = None

        if service_plan_guid == "---ALL--":
            service_plan_guid = None
            
        if org_guid == "---ALL--":
            org_guid = None

        if space_guid == "---ALL--":
            space_guid = None
            
        # define filter function
        def filter_df(r, service_guid = None, service_plan_guid = None, org_guid = None, space_guid = None):
            """ Return True if this row should be hidden
            """
            if service_guid is not None and service_guid != r['service_guid']:
                return True
            if service_plan_guid is not None and service_plan_guid != r['service_plan_guid']:
                return True
            if org_guid is not None and org_guid != r['org_guid']:
                return True
            if space_guid is not None and space_guid != r['org_guid']:
                return True
            return False
        
        # apply filter function on DataFrame, setting field _is_hidden to True or False for each row, as appropriate
        self.services_df['hide_service'] = self.services_df.apply(filter_df, axis = 1, service_guid = service_guid, service_plan_guid = service_plan_guid, org_guid = org_guid, space_guid = space_guid)
 
        # Debug: display number of rows that are hidden/not hidden
        self.debug(self.services_df.groupby(by='hide_service').size())

        # render list of services that are not marked as hidden
        return  """
         <table class="table">
           <thead>
             <tr>
                <th>Service Instance Name</th>
                <th>Service Name</th>
                <th>Service Plan</th>
                <th>Organization</th>
                <th>Space</th>
                <th>Actions</th>
             </tr>
          </thead>
          <tbody>
          {% for row in this.services_df.sort_values(by=['service_instance_name']).itertuples()%}
           {% if row['hide_service'] == False %}
            <tr>
                <td>{{row['service_instance_name']}}</td>
                <td>{{row['service_name']}}</td>
                <td>{{row['service_plan_name']}}</td>
                <td>{{row['org_name']}}</td>
                <td>{{row['space_name']}}</td>
                <td>View credentials</td>
            </tr>
           {% endif %}
          {% endfor %}
          </tbody>
         </table>
        """
    
# --------------------------------------------------------
# instantiate app and run it
app = FetchServiceCredentials()
app.run()
